In [6]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'nvidia-firmware-535-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-doc' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-server-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-headless-390' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-gcc' for glob 'nvidia*'
Note, selecting 'nvidia-headless-418' for glob 'nvidia*'
Note, selecting 'nvidia-headless-430' for glob 'nvidia*'
Note, selecting 'nvidia-headless-435' for glob 'nvidia*'
Note, selecting 'nvidia-headless-440' for glob 'nvidia*'
Note, selecting 'nvidia-headless-450' for glob 'nvidia*'
Note, selecting 'nvidia-headless-455' for glob 'nvidia*'
Note, selecting 'nvidia-headless-460' for glob 'nvidia*'
Note, selecting 'nvidia-headless-465' for glob 'nvidia*'
Note, selecting 'nvidia-headless-470' for glob 'nvidia*'
Note, se

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2024-03-12 12:51:08--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.nvidia.com/downloads/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2024-03-12 12:51:08--  https://developer.nvidia.com/downloads/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?BwDdKs_wq5X1zA58b9jVvK_qVAqtWode0BcRvgz1m-x565QLI7BErUJWEMlhjm6BvD-RLX048ythLw4usSgPCXChXfqr5Dy3OpFhb

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

__device__ void quickSortKernel(int* array, int left, int right) {
    if (left < right) {
        int pivot = array[right];
        int partitionIndex = left;

        for (int i = left; i < right; ++i) {
            if (array[i] < pivot) {
                // Swap array[i] and array[partitionIndex]
                int temp = array[i];
                array[i] = array[partitionIndex];
                array[partitionIndex] = temp;
                partitionIndex++;
            }
        }

        // Swap pivot with array[partitionIndex]
        int temp = array[right];
        array[right] = array[partitionIndex];
        array[partitionIndex] = temp;

        // Recursively sort left and right partitions
        quickSortKernel(array, left, partitionIndex - 1);
        quickSortKernel(array, partitionIndex + 1, right);
    }
}


__global__ void quickSort(int* array, int left, int right) {
    if (left < right) {
        int pivot = array[right];
        int partitionIndex = left;

        for (int i = left; i < right; ++i) {
            if (array[i] < pivot) {
                // Swap array[i] and array[partitionIndex]
                int temp = array[i];
                array[i] = array[partitionIndex];
                array[partitionIndex] = temp;
                partitionIndex++;
            }
        }

        // Swap pivot with array[partitionIndex]
        int temp = array[right];
        array[right] = array[partitionIndex];
        array[partitionIndex] = temp;

        // Recursively sort left and right partitions
        quickSortKernel(array, left, partitionIndex - 1);
        quickSortKernel(array, partitionIndex + 1, right);
    }
}

int main() {
    const int n = 1000; // Size of the array
    int* array_h = (int*)malloc(sizeof(int) * n);

    // Generate random numbers and fill the array
    srand(time(NULL));
    for (int i = 0; i < n; ++i) {
        array_h[i] = rand() % 1000; // Generate random numbers between 0 and 999
    }

    // Print the unsorted array
    printf("Unsorted array:\n");
    for (int i = 0; i < n; ++i) {
        printf("%d ", array_h[i]);
    }
    printf("\n");

    int* array_dev;
    cudaMalloc((void**)&array_dev, n * sizeof(int));
    cudaMemcpy(array_dev, array_h, n * sizeof(int), cudaMemcpyHostToDevice);

    quickSort<<<1, 1>>>(array_dev, 0, n - 1);

    cudaMemcpy(array_h, array_dev, n * sizeof(int), cudaMemcpyDeviceToHost);

    // Print the sorted array
    printf("Sorted array:\n");
    for (int i = 0; i < n; ++i) {
        printf("%d ", array_h[i]);
    }
    printf("\n");

    // Clean up memory and other resources
    free(array_h);
    cudaFree(array_dev);

    return 0;
}

